In [26]:
from annotated_types.test_cases import cases

# from funcNhelpers import detect_saccades, csv_link_img, getEmotion, point_to_aoi
!pip install matplotlib pandas seaborn numpy
!pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd
from pathlib import Path
#Trial
df = pd.read_csv("./PRAKHAR_IIITD/Arousal/28M_NE_C.csv")
print(df.columnsxx)

Index(['Unnamed: 0', 'experiment_id', 'session_id', 'device_id', 'event_id',
       'type', 'device_time', 'logged_time', 'time', 'confidence_interval',
       'delay', 'filter_id', 'eye', 'gaze_x', 'gaze_y', 'gaze_z', 'eye_cam_x',
       'eye_cam_y', 'eye_cam_z', 'angle_x', 'angle_y', 'raw_x', 'raw_y',
       'pupil_measure1', 'pupil_measure1_type', 'pupil_measure2',
       'pupil_measure2_type', 'ppd_x', 'ppd_y', 'velocity_x', 'velocity_y',
       'velocity_xy', 'status', 'participant_id', 'pixel_x', 'pixel_y'],
      dtype='object')


In [27]:
# making life easier by defining data root for simple file handling
DATA=Path("./PRAKHAR_IIITD/")

BBOX_FILE= "Bounding_Box.xlsx"

In [28]:
bbox1= pd.read_excel(BBOX_FILE)
print(bbox1.head())

  label_name  bbox_x  bbox_y  bbox_width  bbox_height    image_name  \
0       Eyes     121     233         260          119  01F_HA_X.BMP   
1      Mouth     151     444         194           90  01F_HA_X.BMP   
2       Nose     164     354         167           87  01F_HA_X.BMP   
3       Eyes     121     233         260          119  02F_AN_C.BMP   
4      Mouth     151     444         194           90  02F_AN_C.BMP   

   image_width  image_height  
0          506           626  
1          506           626  
2          506           626  
3          506           650  
4          506           650  


In [41]:
from pathlib import Path
#also .bmp images can be linked to .csv in dataset, to get corresponding bbox data.
def csv_link_img(csv_file: Path) -> str:
    # print(csv_file.stem + "BMP")
    return csv_file.stem + ".BMP"

def getEmotion(csv_file: Path) -> str:
    if "_AN_" in csv_file.stem:
        return "angry"
    elif "_HA_" in csv_file.stem:
        return "happy"
    elif "_NE_" in csv_file.stem:
        return "neutral"
    return "unknown emotion, ??????"

def detect_saccades(df):
    if "time" in df.columns:
        df = df.sort_values("time")
    dx = df["pixel_x"].diff().abs()
    dy = df["pixel_y"].diff().abs()
    mask = (dx >= 30) | (dy >= 30)
    return df.loc[mask, ["pixel_x", "pixel_y"]].head(5).reset_index(drop=True)

def point_to_aoi(x, y, boxes):
    for i, r in boxes.iterrows():
        x0, y0 = float(r["bbox_x"]), float(r["bbox_y"])
        x1 = x0 + float(r["bbox_width"])
        y1 = y0 + float(r["bbox_height"])
        if x0 <= x <= x1 and y0 <= y <= y1:
            return r["label_name"]
    return "Outside"

In [43]:


aoix= ["Eyes", "Mouth", "Nose"]

saccades=[]
tasks=["Arousal", "Categorisation", "Valence"]

for i in tasks:
    folder_path=DATA/i
    for j in folder_path.glob("*.csv"):
        df = pd.read_csv(j)
        if df[['pixel_x','pixel_y']].dropna(how='any').empty:
            continue
        a=detect_saccades(df)
        if a.shape[0]<1:
            continue

        imgx=csv_link_img(j)
        # print(imgx)
        emotion= getEmotion(j)
        boxes = bbox1[bbox1["image_name"] == imgx]
        a["AOI"]=a.apply(lambda r : point_to_aoi(r["pixel_x"], r["pixel_y"], boxes), axis=1)
        count=a["AOI"].value_counts().reindex(aoix, fill_value=0)
        probs = (count / 5).rename("probability").reset_index().rename(columns={"index":"AOI"})
        probs["task"] = i
        probs["emotion"] = emotion
        probs["file"] = j.stem
        saccades.append(probs)

saccade_df = pd.concat(saccades, ignore_index=True)
saccade_df.head()


,AOI,probability,task,emotion,file
0,Eyes,0.0,Arousal,happy,34M_HA_X
1,Mouth,0.0,Arousal,happy,34M_HA_X
2,Nose,0.6,Arousal,happy,34M_HA_X
3,Eyes,0.6,Arousal,angry,08F_AN_C
4,Mouth,0.0,Arousal,angry,08F_AN_C


In [45]:
output=Path("./saccades_results")
output.mkdir( exist_ok=True)

for i in tasks:
    moose=saccade_df[saccade_df["task"]==i]
    moose.to_csv(output/f"Saccade_{i}.csv", index=False)

print("Saved")


Saved
